In [ ]:
import numpy as np 
import time
import datetime
import os
import sys

import smtplib, ssl
from email.message import EmailMessage

import pyvisa

import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn

In [ ]:
''' MAC ONLY - INTERFACE USING PYVISA - TESTING ONLY '''
# set up measurement devices
rm = pyvisa.ResourceManager()
#pyvisa.log_to_screen()
print(rm)
print(rm.list_resources())
inst = rm.open_resource('GPIB0::13::INSTR')
inst.write('CRDG?')
raw_crdg = inst.read()[:-2]  # use [:-2] to remove the \r\n from the very end
list_crdg = raw_crdg.split(",")  # split string into a list, delimiter is a comma
print(list_crdg)

In [ ]:
''' LINUX ONLY - INTERFACE USING LINUX-GPIB '''
import Gpib

inst = Gpib.Gpib(0,13) # Device address 0
inst.write("CRDG?")
#print(inst.read(1000))
list_gpib = str(inst.read(1000))[2:-5].split(",")
print(list_gpib)

In [ ]:
'''
    measure temperature using GPIB via an Agilent 82357A USB/GPIB converter
    this converter is hooked up to the raspberry pi and the LakeShore Model 
    218 temperature monitor.
    
    this is written using the LINUX-GPIB library since it will be the final
    code run by the raspberry pi, but I added a comment for the mac version
'''

def T_measure(channel_num=1):    
    '''
        INPUTS
            channel_num - [integer] -  which channel of the temp monitor to read. should be between 1 and 8.
        OUTPUTS
            gpm - [float] - gallons per minute readout value from the sensor.
    '''
    # LINUX
    inst = Gpib.Gpib(0,13) # Device address 0
    
    # MAC
    #inst = rm.open_resource('GPIB0::13::INSTR')

    # both linux & mac
    inst.write('CRDG?')  # query celsius reading
    raw_crdg = str(inst.read())[2:-5]  # use [:-2] to remove the \r\n from the very end
    list_crdg = raw_crdg.split(",")  # split string into a list, delimiter is a comma
    return list_crdg[channel_num-1]  # first entry in array is 0, so channel 1 is entry 0


In [ ]:
''' 
    measure gallons per minute using an analog-digital converter
    ADC measures the voltage across a resistive load and converts
    it into a digital signal for the raspberry pi to read in mV's
    ADS1115 documentation at https://docs.circuitpython.org/projects/ads1x15/en/latest/ 
'''
def GPM_measure(resistor_load=100):
    '''
        INPUTS
            resistor_load - [float] - load resistor ohm value. by default should be 100 ohms.
        OUTPUTS
            gpm - [float] -  gallons per minute readout value from the sensor. converted from ADC measurement
    '''
    # Create the I2C bus
    i2c = busio.I2C(board.SCL, board.SDA)

    # Create the ADC object using the I2C bus
    ads = ADS.ADS1115(i2c)

    # Create single-ended input on channel 0
    chan = AnalogIn(ads, ADS.P0, ADS.P1)

    print("{:>5}\t{:>5}".format('raw', 'v'))  # print headers for table of measurements
    
    ''' perform measurement using ADC and gpib '''
    # use try-except to allow pi to continue working even if error is raised
    try:
        raw_data = chan.value
        voltage = chan.voltage
    except:
        pass
        
    # take measurement and print
    print("{:>5}\t{:>5.3f}".format(raw_data, voltage))

    ''' convert measurement '''
    # convert mA measurement to flowrate
    ### range of device is from 4mA to 20mA
    ### gpm is from ASP to AEP, by default 0.00 to 5.28 gpm

    
    #changed SSP and SEP to be function parameters
    #SSP = 4     # signal start point [mV]
    #SEP = 20    # signal end point [mV]
    ASP = 0.00   # analog start point [gpm]
    AEP = 5.28   # analog end point [gpm]

    # shift data down by 4mV (SSP), then 
    # divide by 16mV (SEP-SSP) to get percentage. 
    # then convert to gpm using the meter's analog range
    #percentage = (data-SSP)/(SEP-SSP)
    #gpm = percentage*(AEP-ASP)
    
    return 

while True:
    time.sleep(2)
    print(GPM_measure())

  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None
  raw	    v
    0	0.000
None


In [ ]:
'''
    send text messages on reading failure to the 
    phone numbers listed below in the phonebook.
'''
'''
phonebook = {
                "Jorge" : '2403054216@tmomail.net',
                "Manuel" : '7203520897@tmomail.net',
                "Pete" : '3034781436@vtext.com',
            }
'''

phonebook = {
                "Jorge" : '2403054216@tmomail.net',
            }

def Alert_protocol(alert, temp_reading, flow_reading, recipients=phonebook):
    '''
        INPUTS
            alert -  specific message to be sent regarding alert
            temp_reading -  float returned from T_measure()
            flow_reading -  float returned from GPM_measure()
            recipients - python dict with values as phone number emails 
        OUTPUTS
            none
    '''
    
    # write contents of email, use string formatting to insert time and data
    time_report = datetime.datetime.now().strftime("%H:%M:%S %p on %h %d %Y") 
    contents = """ \n
    Triggered at {} \n
    Alert: {} \n
    Temperature: {:1.2f} K \n
    Flow rate: {:1.2f} GPM """.format(time_report, alert, temp_reading, flow_reading)
    
    # create emailmessage object with subject, source, contents, and recipients
    msg = EmailMessage()
    msg['Subject'] = "Freeze's Alarm Triggered \n"
    msg['From'] = "sce_freezepi@nist.gov"
    
    if type(recipients) == dict:  # convert possible dict to string
        msg['To'] = list(recipients.values())
    else:  
        msg['To'] = recipients
        
    msg.set_content(contents)
                    
    # create smtp object with nist smtp server and then send
    smtpObj = smtplib.SMTP('smtp.nist.gov')
    try: 
        smtpObj.send_message(msg)   
        print("Successfully sent email.")
    except:
        print("Error occurred.")
        
    return 

# helper function
def write_to_log(data_path, filename, string):
    # record to .txt file
    with open(os.path.join(data_path, filename), 'a') as file:  # use append mode to... append to today's file
        print(string)  # record in console
        file.write(string)  # record in text file
        
    return

Alert_protocol("Testing", -1, -1, recipients=phonebook["Jorge"])  # test

In [ ]:
''' MAIN CODE FOR RASPBERRY PI
loop methodology
1) check temp & flow readings
2) report bad readings via text
3) log to file and wait x minutes
'''

last_alert = None  # init last alert as None so we know if it's booting or sending a real alert

''' establish current directory and logging file '''
cwd = os.getcwd()  # same folder as script, should be ~/GitHub/<this repo>
data_path = os.path.join(cwd, "Freeze_Logs")  # dir for saving .txt logs

# create directory if data folder is missing   
if not os.path.exists(data_path):
    os.makedirs(data_path)

# create text file with today's date as name
today = datetime.date.today()
filename = today.strftime("%m_%d_%y") + ".txt"   # e.g. 09_13_22.txt

with open(os.path.join(data_path, filename), 'a') as f:  # use append mode in case file exists already
    date = str(datetime.datetime.now().strftime("%D %I:%M:%S %p"))  #Month-Day-Yr  Hr:Min:Sec am/pm
    f.write('Initializing Freeze\'s data collection: ' + date + '\n')
        
''' begin main loop '''
while True:
    ''' step (1) check flow & temp reading '''
    try: 
        temp_reading = T_measure() 
    except:
        temp_reading = -1
        print("Temperature reading failed.")
    
    try:
        flow_reading = GPM_measure()
    except:
        flow_reading = -1
        print("Flow reading failed.")
    
    
    
    ''' step (2): report bad readings via text'''
    if flow_reading or temp_reading == -1:
        if last_alert is not None: # on boot, last_alert == None
            print("Activating bad reading alert protocol.")
            Alert_protocol(alert="Bad flow/temp reading", flow_reading=flow_reading, temp_reading=temp_reading, recipients=phonebook["Jorge"])  # only I shall know my mistakes
        
    elif flow_reading <= flow_threshold or temp_reading <= temp_threshold:
        # check if 1 hour has passed
        hours_passed = (datetime.datetime.now() - last_alert).seconds/3600
        if hours_passed >= 1:
            print("Activating threshold alert protocol.")
            Alert_protocol(alert="Threshold triggered.", flow_reading=flow_reading, temp_reading=temp_reading, recipients=phonebook)
            last_alert = datetime.datetime.now()  # update datetime object
        else:
            print("Alarm triggered, but no email sent")
            
    flow_reading, flow_threshold = 2.2, 1.5
    
    temp_reading, temp_threshold = 8, 5
    
                
    ''' step (3): log data to file and wait'''
    timestamp = str(datetime.datetime.now().strftime("%I:%M:%S %p")) # Hr:Min:Sec am/pm
    
    if flow_reading == -1 or temp_reading == -1: 
        log_message = "[{}] ||| {:1.2f} K ||| {:1.2f} GPM ||| BAD READING \n".format(timestamp, temp_reading, flow_reading)

    elif flow_reading <= flow_threshold or temp_reading <= temp_threshold:
        log_message = "[{}] ||| {:1.2f} K ||| {:1.2f} GPM ||| ALARM TRIGGERED!! Temp_Threshold = {} K and Flow_Threshold = {}GPM\n".format(timestamp, temp_reading, \
                                                                                                                                    flow_reading, temp_threshold, flow_threshold)

    else:
        log_message = "[{}] ||| {:1.2f} K ||| {:1.2f} GPM \n".format(timestamp, temp_reading, flow_reading)
        
    write_to_log(data_path, filename, log_message)
    
    time.sleep(10)  # wait 30 minutes before measuring again

In [ ]:
timestamp = str(datetime.datetime.now().strftime("%I:%M:%S %p")) # Hr:Min:Sec am/pm
data_measurement_string = "[{0}] ||| {1:1.3f} K ||| {1:1.3f} GPM  \n".format(timestamp, -1, -1)

print(data_measurement_string)  # record in console
